In [1]:
import optuna
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm import tqdm
from lightgbm import log_evaluation

import warnings
warnings.filterwarnings('ignore')

/Users/idris/miniconda3/envs/forecast/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def calculate_lags(df, col, lags, shift=0):
    for l in lags:
        df[f"lag_{col}_{l}"] = df.groupby(["ts_id"])[col].shift(shift + l).astype(np.float16)

def calculate_rollings(df, col, rollings, shift=0):
    for r in rollings:
        df[f"rol_mean_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).mean()
        df[f"rol_std_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).std()
        df[f"rol_min_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).min()
        df[f"rol_max_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).max()


from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [3]:
path = '/Users/idris/Documents/ds_project/forecast_store_sales/data/'
cutoff = '2017-07-31'

col_to_ignore = ["date", "is_future", "forecast_step", "ts_id"]

TARGET_COL = 'sales'

prediction_length = 16
lags_target=list(range(1, 16))
rollings_target=[2, 4, 8, 16]

In [4]:
df_cutoff = pd.read_csv(path + f'fe/cutoff/{cutoff}/final_frame.csv', sep=';').drop('transactions', axis=1)
df_cutoff['date'] = pd.to_datetime(df_cutoff['date'])
df_cutoff['is_future'] = df_cutoff['is_future'].fillna(False)
df_cutoff["time_idx"] = ((df_cutoff["date"] - df_cutoff["date"].min()).dt.days).astype(int)

df_cv = df_cutoff.loc[df_cutoff["is_future"] == False].reset_index(drop=True)

clean = pd.read_csv(path + f'trainclean.csv', sep=';')
clean['date'] = pd.to_datetime(clean['date'])

In [5]:
prediction_length = 16

lags_target=list(range(1, 16))
rollings_target=[2, 4, 8, 16]

In [6]:
def objective(trial):
     
    params = {
        "objective": "tweedie",
        "tweedie_variance_power": trial.suggest_float("tweedie_variance_power", 1.1, 1.6, step=0.1),
        "n_estimators": trial.suggest_categorical("n_estimators", [1000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
        "random_state": 1990,
        "metric": "rmse",
        "verbose": -1
    }
    
    list_wape = []
    
    for train_idx, test_idx in tqdm(GroupTimeSeriesSplit().split(df_cv, groups=df_cv['time_idx'])):
    
        i_train = df_cv.loc[train_idx, 'time_idx'].unique()
        df_train = df_cv.loc[df_cv['time_idx'].isin(i_train)].drop(columns=col_to_ignore)
        i_test = df_cv.loc[test_idx, 'time_idx'].unique()[:16]
        df_test = df_cv.loc[df_cv['time_idx'].isin(i_test)].drop(columns=col_to_ignore).drop(columns=TARGET_COL) 

        min_train_idx = df_test['time_idx'].min() - prediction_length
        max_train_idx = df_test['time_idx'].max()
        cutoff_idx = np.arange(min_train_idx, max_train_idx + 1, 1)
        df_rec = df_cv.loc[df_cv['time_idx'].isin(cutoff_idx)]    

        df_rec.loc[df_rec['time_idx'].isin(i_test), 'is_future'] = True
        df_rec.loc[df_rec["is_future"] == True, "sales"] = np.nan
                
        print('id time to pred', i_test)
        
        train_ds = lgb.Dataset(
        data=df_train.drop(columns=TARGET_COL), 
        label=df_train[TARGET_COL]
        )

        gbm = lgb.train(
        params=params,
        train_set=train_ds,
        valid_sets=train_ds,
        callbacks=[log_evaluation(period=100)]
        )

        first_future_time_idx = i_test.min()
        df_rec.loc[:, "forecast_step"] = df_rec.loc[:, "time_idx"] - first_future_time_idx + 1 

        for fs in range(1, prediction_length + 1):

            # predict one step
            df_predict = df_rec.loc[df_rec["forecast_step"] == fs].drop(columns=col_to_ignore + [TARGET_COL])
            predictions = np.clip(gbm.predict(df_predict), a_min=0, a_max=None)

            # update sales_quantity usings predictions
            df_rec.loc[df_rec["forecast_step"] == fs, "sales"] = predictions

            # update target lags & rollings
            calculate_lags(df_rec, TARGET_COL, lags_target)
            calculate_rollings(df_rec, TARGET_COL, rollings_target)

        df_forecast = df_rec.loc[df_rec["is_future"] == True, ["ts_id", "date", "sales"]]
        df_forecast["forecast"] = df_forecast["sales"].round(0).astype(np.float16)
        df_forecast.drop(columns="sales", inplace=True)

        df_error = pd.merge(df_forecast, clean[['ts_id', 'date', 'sales']], how="left", on=['ts_id', 'date'])
        wape = np.round(np.sum(np.abs(df_error["sales"] - df_error["forecast"])) / np.sum(df_error["sales"]), 3)
        print('wape: ', wape)
        list_wape.append(wape)
    
    mean_wape = np.mean(list_wape)
    
    return mean_wape

In [7]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-21 19:59:02,193] A new study created in memory with name: no-name-429f9755-890f-4da9-8507-1e4ca235bc20
0it [00:00, ?it/s]

id time to pred [283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298]
[100]	training's rmse: 199.458
[200]	training's rmse: 179.281
[300]	training's rmse: 170.337
[400]	training's rmse: 164.152
[500]	training's rmse: 160.042
[600]	training's rmse: 156.07
[700]	training's rmse: 152.984
[800]	training's rmse: 150.567
[900]	training's rmse: 148.103
[1000]	training's rmse: 146.087


1it [00:40, 40.18s/it]

wape:  0.137
id time to pred [561 562 563 564 565 566 567 568 569 570 571 572 573 574 575 576]
[100]	training's rmse: 214.817
[200]	training's rmse: 192.429
[300]	training's rmse: 180.217
[400]	training's rmse: 171.586
[500]	training's rmse: 165.748
[600]	training's rmse: 160.597
[700]	training's rmse: 156.304
[800]	training's rmse: 153.056
[900]	training's rmse: 150.033
[1000]	training's rmse: 147.087


2it [01:44, 54.37s/it]

...

wape:  0.254
id time to pred [839 840 841 842 843 844 845 846 847 848 849 850 851 852 853 854]
[100]	training's rmse: 291.909
[200]	training's rmse: 250.451
[300]	training's rmse: 234.934
[400]	training's rmse: 224.983
[500]	training's rmse: 217.02
[600]	training's rmse: 210.927
[700]	training's rmse: 206.475
[800]	training's rmse: 202.273
[900]	training's rmse: 198.669
[1000]	training's rmse: 195.552


3it [03:11, 71.08s/it]

wape:  0.21
id time to pred [1117 1118 1119 1120 1121 1122 1123 1124 1125 1126 1127 1128 1129 1130
 1131 1132]
[100]	training's rmse: 299.624
[200]	training's rmse: 256.367
[300]	training's rmse: 241.194
[400]	training's rmse: 232.527
[500]	training's rmse: 225.727
[600]	training's rmse: 220.113


3it [05:01, 100.54s/it]


KeyboardInterrupt: 

In [ ]:
print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
 Trial 0 finished with value: 0.14940000000000003 and parameters: 

{
    'tweedie_variance_power': 1.4000000000000001,
    'n_estimators': 500,
    'learning_rate': 0.067307721852165,
    'num_leaves': 300,
    'max_depth': 10,
    'min_data_in_leaf': 1300,
    'lambda_l1': 100,
    'lambda_l2': 80,
    'min_gain_to_split': 9.72024574054834,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'feature_fraction': 0.6000000000000001
}

. Best is trial 0 with value: 0.14940000000000003.

In [ ]:
Trial 7 finished with value: 0.156 and parameters: {'tweedie_variance_power': 1.3, 'n_estimators': 500, 'learning_rate': 0.047505552909073004, 'num_leaves': 2780, 'max_depth': 9, 'min_data_in_leaf': 4400, 'lambda_l1': 65, 'lambda_l2': 40, 'min_gain_to_split': 9.020620157726453, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'feature_fraction': 0.9}. Best is trial 7 with value: 0.156.

In [ ]:
Trial 0 finished with value: 0.15299999999999997 and parameters: {'tweedie_variance_power': 1.4000000000000001, 'n_estimators': 500, 'learning_rate': 0.06908883769088184, 'num_leaves': 600, 'max_depth': 10, 'min_data_in_leaf': 400, 'lambda_l1': 5, 'lambda_l2': 50, 'min_gain_to_split': 3.4052603539384463, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 0 with value: 0.15299999999999997.

In [25]:
df_cutoff[df_cutoff['ts_id'] == '9_28'].sort_values('date', ascending=False).head(20)

,ts_id,date,id,bu,famid,sales,onprom,lag_sales_1,lag_sales_2,lag_sales_3,...,hol_before_1,hol_before_2,hol_before_3,hol_after_1,prixoil,isclosed,typeid,cityid,stateid,cluster
2444481,9_28,2017-08-15,3000883,9,28,377.118167,0,379.25,527.50,448.00,...,0,0,0,1,47.57,0,1,0,0,6
2444480,9_28,2017-08-14,2999101,9,28,379.155766,0,527.50,448.00,467.00,...,0,0,0,1,47.59,0,1,0,0,6
2444479,9_28,2017-08-13,2997319,9,28,527.731250,0,448.00,467.00,341.00,...,0,0,1,0,48.81,0,1,0,0,6
2444478,9_28,2017-08-12,2995537,9,28,447.991523,0,467.00,341.00,327.00,...,0,1,0,0,48.81,0,1,0,0,6
2444477,9_28,2017-08-11,2993755,9,28,467.101870,22,341.00,327.00,358.25,...,1,0,0,0,48.81,0,1,0,0,6
2444476,9_28,2017-08-10,2991973,9,28,341.112831,0,327.00,358.25,375.25,...,0,0,0,0,48.54,0,1,0,0,6
2444475,9_28,2017-08-09,2990191,9,28,327.100216,1,358.25,375.25,614.50,...,0,0,0,1,49.59,0,1,0,0,6
2444474,9_28,2017-08-08,2988409,9,28,358.329020,0,375.25,614.50,497.00,...,0,0,1,0,49.07,0,1,0,0,6
2444473,9_28,2017-08-07,2986627,9,28,375.205631,0,614.50,497.00,544.00,...,0,1,0,0,49.37,0,1,0,0,6
2444472,9_28,2017-08-06,2984845,9,28,614.312139,0,497.00,544.00,364.50,...,1,0,0,0,49.57,0,1,0,0,6


In [36]:
prediction_length = 8
lags_target=list(range(1, 16))
rollings_target=[2, 4, 8, 16]

In [33]:
def objective(trial):

    def calculate_lags(df, col, lags, shift=0):
        for l in lags:
            df[f"lag_{col}_{l}"] = df.groupby(["ts_id"])[col].shift(shift + l).astype(np.float16)

    def calculate_rollings(df, col, rollings, shift=0):
        for r in rollings:
            df[f"rol_mean_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).mean()
            df[f"rol_std_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).std()
            df[f"rol_min_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).min()
            df[f"rol_max_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).max()


    params = {
        "boosting_type": "gbdt", 
        "num_leaves": trial.suggest_int("num_leaves", 4, 10),
        "max_depth": -1, 
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1), 
        "n_estimators": 500, #100
        "subsample_for_bin": 200000, 
        "objective": trial.suggest_categorical("objective", ["regression", "tweedie"]),
        "class_weight": None, 
        "min_split_gain": 0.0,
        "min_child_weight": 0.001, 
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 100, step=10),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0, step=0.1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0, step=0.1),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1.0, step=0.1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1.0, step=0.1),
        "random_state": 666,
        "metric": "rmse"
    }
    
    params["num_leaves"] = 2**params["num_leaves"]-1
    
    if params["objective"] == "tweedie":
        params["tweedie_variance_power"] = trial.suggest_float("tweedie_variance_power", 1.0, 1.9, step=0.1)
    
    if params["subsample"] < 1.0:
        params["subsample_freq"] = trial.suggest_int("subsample_freq", 1, 7)
    


    train_ds = lgb.Dataset(
        data=df_train.drop(columns=TARGET_COL), 
        label=df_train[TARGET_COL]
    )

    print("Fit")
    gbm = lgb.train(
        params=params,
        train_set=train_ds,
        valid_sets=train_ds
        )

    df_cutoff.loc[df_cutoff["is_future"] == True, "sales"] = np.nan

    for fs in range(1, prediction_length + 1):
        print(fs)

        # predict one step
        df_predict = df_cutoff.loc[df_cutoff["forecast_step"] == fs].drop(columns=col_to_ignore + [TARGET_COL])
        predictions = np.clip(gbm.predict(df_predict), a_min=0, a_max=None)

        # update sales_quantity usings predictions
        df_cutoff.loc[df_cutoff["forecast_step"] == fs, "sales"] = predictions

        # update target lags & rollings
        calculate_lags(df_cutoff, TARGET_COL, lags_target)
        calculate_rollings(df_cutoff, TARGET_COL, rollings_target)
    
    print("Format")
    df_forecast = df_cutoff.loc[df_cutoff["is_future"] == True, ["ts_id", "date", "sales"]]
    df_forecast["forecast"] = df_forecast["sales"].round(0).astype(np.float16)
    df_forecast.drop(columns="sales", inplace=True)
    print("Log metrics")
    df_error = pd.merge(df_forecast, clean[['ts_id', 'date', 'sales']], how="left", on=['ts_id', 'date'])
    wape = np.round(np.sum(np.abs(df_error["sales"] - df_error["forecast"])) / np.sum(df_error["sales"]), 4)
    
    return wape

In [37]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-14 20:33:22,518] A new study created in memory with name: no-name-3593a23c-3b2f-4436-ba0e-ae10dd20e2d4
/Users/idris/miniconda3/envs/forecast/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  in terms of current number of round (e.g. yields learning rate decay).


Fit
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.126711 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 35821
[LightGBM] [Info] Number of data points in the train set: 2244863, number of used features: 175
[LightGBM] [Info] Start training from score 6.111432
[1]	training's rmse: 1187.75
[2]	training's rmse: 1169.07
[3]	training's rmse: 1156.22
[4]	training's rmse: 1144.68
[5]	training's rmse: 1133.28
[6]	training's rmse: 1121.62
[7]	training's rmse: 1109.61
[8]	training's rmse: 1097.05
[9]	training's rmse: 1084.02
[10]	training's rmse: 1070.5
[11]	training's rmse: 1056.41
[12]	training's rmse: 1041.96
[13]	training's rmse: 1026.99
[14]	training's rmse: 1011.46
[15]	training's rmse: 995.469
[16]	training's rmse: 979.022
[17]	training's rmse: 962.064
[18]	training's rmse: 944.743
[19]	training's rmse: 927.005
[20]	training's rmse: 9

[I 2022-05-14 20:36:14,107] Trial 0 finished with value: 0.0778 and parameters: {'num_leaves': 6, 'learning_rate': 0.07998230745890325, 'objective': 'tweedie', 'min_child_samples': 30, 'subsample': 0.8, 'colsample_bytree': 0.5, 'reg_alpha': 0.2, 'reg_lambda': 0.30000000000000004, 'tweedie_variance_power': 1.9, 'subsample_freq': 4}. Best is trial 0 with value: 0.0778.
/Users/idris/miniconda3/envs/forecast/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  in terms of current number of round (e.g. yields learning rate decay).


Fit
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.148889 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 35821
[LightGBM] [Info] Number of data points in the train set: 2244863, number of used features: 175
[LightGBM] [Info] Start training from score 450.984147
[1]	training's rmse: 1147.2
[2]	training's rmse: 1069.91
[3]	training's rmse: 998.149
[4]	training's rmse: 931.649
[5]	training's rmse: 870.393
[6]	training's rmse: 814.228
[7]	training's rmse: 762.883
[8]	training's rmse: 715.519
[9]	training's rmse: 672.213
[10]	training's rmse: 632.737
[11]	training's rmse: 596.649
[12]	training's rmse: 563.454
[13]	training's rmse: 533.283
[14]	training's rmse: 506.147
[15]	training's rmse: 480.663
[16]	training's rmse: 458.138
[17]	training's rmse: 437.556
[18]	training's rmse: 419.321
[19]	training's rmse: 402.492
[20]	training's rmse:

[I 2022-05-14 20:38:57,857] Trial 1 finished with value: 0.0746 and parameters: {'num_leaves': 6, 'learning_rate': 0.07513117103691569, 'objective': 'regression', 'min_child_samples': 10, 'subsample': 1.0, 'colsample_bytree': 0.5, 'reg_alpha': 0.0, 'reg_lambda': 0.1}. Best is trial 1 with value: 0.0746.
/Users/idris/miniconda3/envs/forecast/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  in terms of current number of round (e.g. yields learning rate decay).


...

Fit


/Users/idris/miniconda3/envs/forecast/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  in terms of current number of round (e.g. yields learning rate decay).


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.105995 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 35821
[LightGBM] [Info] Number of data points in the train set: 2244863, number of used features: 175
[LightGBM] [Info] Start training from score 6.111432
[1]	training's rmse: 1184.5
[2]	training's rmse: 1135.45
[3]	training's rmse: 1083.39
[4]	training's rmse: 1028.32
[5]	training's rmse: 970.408
[6]	training's rmse: 911.484
[7]	training's rmse: 852.443
[8]	training's rmse: 794.123
[9]	training's rmse: 737.855
[10]	training's rmse: 685.171
[11]	training's rmse: 636.634
[12]	training's rmse: 591.304
[13]	training's rmse: 550.675
[14]	training's rmse: 515.293
[15]	training's rmse: 483.273
[16]	training's rmse: 455.306
[17]	training's rmse: 431.111
[18]	training's rmse: 410.84
[19]	training's rmse: 392.684
[20]	training's rmse: 376.83

[I 2022-05-15 00:13:41,261] Trial 19 finished with value: 0.0776 and parameters: {'num_leaves': 5, 'learning_rate': 0.1190962206280641, 'objective': 'tweedie', 'min_child_samples': 40, 'subsample': 0.5, 'colsample_bytree': 0.4, 'reg_alpha': 0.6000000000000001, 'reg_lambda': 0.2, 'tweedie_variance_power': 1.3, 'subsample_freq': 5}. Best is trial 2 with value: 0.0691.


Number of finished trials: 20
Best trial:
  Value: 0.0691
  Params: 
    num_leaves: 6
    learning_rate: 0.03309337787385885
    objective: tweedie
    min_child_samples: 20
    subsample: 0.7000000000000001
    colsample_bytree: 0.4
    reg_alpha: 0.30000000000000004
    reg_lambda: 0.1
    tweedie_variance_power: 1.1
    subsample_freq: 6


In [42]:
trial.datetime_complete

datetime.datetime(2022, 5, 14, 20, 41, 54, 706093)